In [1]:
import os
import pdb
import torch
import pandas
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv
import torch.nn.functional as F

## CORA

In [2]:
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())

print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Has isolated nodes: False
Has self-loops: False
Is undirected: True


In [3]:
class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        torch.manual_seed(1234567)
        
        self.conv1 = GATConv(dataset.num_features, hidden_channels, dropout=0.6)
        self.conv2 = GATConv(hidden_channels, dataset.num_classes, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [4]:
model = GAT(hidden_channels=8, heads=8)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out = model(data.x, data.edge_index)  # Perform a single forward pass.
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss

def test(mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    correct = pred[mask] == data.y[mask]  # Check against ground-truth labels.
    acc = int(correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
    return acc

for epoch in range(1, 101):
    loss = train()
    val_acc = test(data.val_mask)
    test_acc = test(data.test_mask)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

GAT(
  (conv1): GATConv(1433, 8, heads=1)
  (conv2): GATConv(8, 7, heads=1)
)
Epoch: 001, Loss: 1.9452, Val: 0.1740, Test: 0.1660
Epoch: 002, Loss: 1.9426, Val: 0.2100, Test: 0.1950
Epoch: 003, Loss: 1.9420, Val: 0.2360, Test: 0.2200
Epoch: 004, Loss: 1.9405, Val: 0.3500, Test: 0.3250
Epoch: 005, Loss: 1.9370, Val: 0.4160, Test: 0.4220
Epoch: 006, Loss: 1.9393, Val: 0.4040, Test: 0.4380
Epoch: 007, Loss: 1.9403, Val: 0.3980, Test: 0.4460
Epoch: 008, Loss: 1.9349, Val: 0.4080, Test: 0.4380
Epoch: 009, Loss: 1.9329, Val: 0.4620, Test: 0.4650
Epoch: 010, Loss: 1.9200, Val: 0.5220, Test: 0.5220
Epoch: 011, Loss: 1.9297, Val: 0.5400, Test: 0.5350
Epoch: 012, Loss: 1.9223, Val: 0.5780, Test: 0.5640
Epoch: 013, Loss: 1.9300, Val: 0.5980, Test: 0.5720
Epoch: 014, Loss: 1.9151, Val: 0.6080, Test: 0.5830
Epoch: 015, Loss: 1.9179, Val: 0.6040, Test: 0.5820
Epoch: 016, Loss: 1.9188, Val: 0.6120, Test: 0.5850
Epoch: 017, Loss: 1.9023, Val: 0.5900, Test: 0.5730
Epoch: 018, Loss: 1.9084, Val: 0.6000,

In [5]:
test_acc = test(data.val_mask)
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.7400
